# Практическое занятие по методам трекинга объектов на основе модели Yolo

## Подготовка окружения для работы с Yolo

Установка пакета Ultralytics для работы с моделью Yolo.

In [ ]:
!pip install ultralytics

Загрузка весовых коэффициентов предобученной модели.

In [ ]:
!wget https://github.com/ultralytics/assets/releases/download/v8.1.0/yolov8x.pt

In [ ]:
!ls

## Детекция объектов на изображении

In [ ]:
%%writefile sources.csv
url
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_1_fixed/image_001.jpg
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_1_fixed/image_002.jpg
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_1_fixed/image_003.jpg
https://raw.githubusercontent.com/ant-nik/neural_network_course/main/practice_2_data/video_2_fixed/image_005.jpg

In [ ]:
!for line in `cat sources.csv`; do wget $line; done

In [ ]:
%%writefile localfiles.csv
image_001.jpg
image_002.jpg
image_003.jpg
image_005.jpg

In [ ]:
from ultralytics import YOLO
import plotly.express as plte

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x.pt')
# Define remote image or video URL
source = 'localfiles.csv'
# Run inference on the source
results = model(".")  # list of Results objects
for res in results:
    plte.imshow(res.plot()).show()